In [1]:
import random

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, log_loss

from src.dataset import get_dataset
from src.models.dualemb import DualEmbPredictor
from src.models.elog import ELOgPredictor
from src.models.frequency_random import FrequencyRandomMatchPredictor
from src.models.uniform_random import UniformRandomMatchPredictor

In [2]:
ITERATIONS = 1
random.seed(5438)
np.random.seed(5438)

In [3]:
def determine_target(row):
    if row["home_score"] > row["away_score"]:
        return 0
    elif row["home_score"] == row["away_score"]:
        return 1
    else:
        return 2

In [4]:
dataset = get_dataset()

/mnt/c/Personal/football-matches-prediction/src/dataset.py:47: DtypeWarning: Columns (11,14,50,102,123,125,129,131,133,135,137,139,141,143,145,147,149,157,158,160,161,163,165,167,169,171,173,184,186,187,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  mls_df = pd.read_csv("data/mls_matches.csv")


In [5]:
dataset

,date,home_team,home_score,away_score,away_team,neutral,fold
0,1971-08-07,Santa Cruz,1,4,Corinthians,False,brazil
1,1971-08-07,Internacional,0,0,Fluminense,False,brazil
2,1971-08-07,Palmeiras,1,0,Portuguesa-SP,False,brazil
3,1971-08-07,Ceará,0,0,Vasco da Gama,False,brazil
4,1971-08-07,Coritiba,0,2,Cruzeiro,False,brazil
...,...,...,...,...,...,...,...
143000,2017-07-06,Grenada,0,2,Barbados,False,international
143001,2016-08-25,Barawa,0,5,Tamil Eelam,False,international
143002,2016-08-26,Barawa,2,3,Chagos Islands,False,international
143003,2016-08-28,Chagos Islands,1,5,Tamil Eelam,True,international


In [6]:
model_classes = [
    # FrequencyRandomMatchPredictor,
    # UniformRandomMatchPredictor,
    # ELOgPredictor,
    DualEmbPredictor,
]
# folds_names = ["brazil", "libertadores", "mls", "europe", "international"]
folds_names = ["brazil", "international"]

In [7]:
folds_train = [dataset[dataset["fold"] != name] for name in folds_names]
folds_test = [dataset[dataset["fold"] == name] for name in folds_names]

In [8]:
results = pd.DataFrame({}, columns=["metric", "model", "fold", "iteration", "value"])

In [9]:
for iteration in range(ITERATIONS):
    for model_class in model_classes:
        for fold_train, fold_test, fold_test_name in zip(
            folds_train, folds_test, folds_names
        ):
            model = model_class()
            model.fit(fold_train)
            pred = model.predict_proba(fold_test)
            max_pred = np.argmax(pred, axis=1)
            target = fold_test.apply(determine_target, axis=1).to_numpy()
            report = classification_report(
                target, max_pred, target_names=["win", "draw", "loss"], output_dict=True
            )
            metrics = {
                "accuracy": report["accuracy"],
                "log_loss": log_loss(target, pred, labels=[0, 1, 2]),
                "win_precision": report["win"]["precision"],
                "win_recall": report["win"]["recall"],
                "draw_precision": report["draw"]["precision"],
                "draw_recall": report["draw"]["recall"],
                "loss_precision": report["loss"]["precision"],
                "loss_recall": report["loss"]["recall"],
                "weighted_precision": report["weighted avg"]["precision"],
                "weighted_recall": report["weighted avg"]["recall"],
            }
            for key, value in metrics.items():
                results.loc[len(results)] = {
                    "metric": key,
                    "model": model_class.__name__,
                    "fold": fold_test_name,
                    "iteration": iteration + 1,
                    "value": value,
                }

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 118362/118362 [03:48<00:00, 518.36it/s, loss=2.2542]
/home/razevedo/.cache/pypoetry/virtualenvs/football-matches-prediction-ljJPvyK7-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/razevedo/.cache/pypoetry/virtualenvs/football-matches-prediction-ljJPvyK7-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/

In [10]:
results

,metric,model,fold,iteration,value
0,accuracy,DualEmbPredictor,brazil,1,0.308607
1,log_loss,DualEmbPredictor,brazil,1,1.101856
2,win_precision,DualEmbPredictor,brazil,1,0.474359
3,win_recall,DualEmbPredictor,brazil,1,0.287160
4,draw_precision,DualEmbPredictor,brazil,1,0.000000
5,draw_recall,DualEmbPredictor,brazil,1,0.000000
6,loss_precision,DualEmbPredictor,brazil,1,0.240511
7,loss_recall,DualEmbPredictor,brazil,1,0.705102
8,weighted_precision,DualEmbPredictor,brazil,1,0.286329
9,weighted_recall,DualEmbPredictor,brazil,1,0.308607


In [11]:
results.groupby(["metric", "model", "fold"])["value"].mean().reset_index().groupby(
    ["metric", "model"]
)["value"].mean().reset_index().pivot(index="model", columns="metric", values="value")

metric,accuracy,draw_precision,draw_recall,log_loss,loss_precision,loss_recall,weighted_precision,weighted_recall,win_precision,win_recall
model,,,,,,,,,,
ELOgPredictor,0.425599,0.0,0.0,1.140806,0.263635,0.27378,0.305239,0.425599,0.485611,0.727405
FrequencyRandomMatchPredictor,0.485622,0.0,0.0,1.055025,0.000000,0.00000,0.235850,0.485622,0.485622,1.000000
UniformRandomMatchPredictor,0.485622,0.0,0.0,1.098612,0.000000,0.00000,0.235850,0.485622,0.485622,1.000000
